In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

load_dotenv()
llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
from guardrails import Guard
from guardrails.hub import DetectPII
from guardrails.hub import ToxicLanguage

guard = Guard().use(
    DetectPII(
        pii_entities = ['EMAIL', 'MOBILE', 'ADDRESS'],
        on_fail = 'exception'
    )
)
print('Guard Created with DetectPII validator')

toxic_guard = Guard().use(ToxicLanguage(threshold=0.5, on_fail='exception'))

In [7]:
def call_llm_guardrails(user_input: str):
    """ 
    Call LLM with input and output guardrails. 
    """
    print(f"\n{'='*60}")
    print(f"User Input: {user_input}")
    print(f"{'='*60}")

    try:
        toxic_guard.validate(user_input)
        print('Input passed toxicity check')
    except Exception as ex:
        result = 'INPUT BLOCKED: Toxic content detected'
        print(result)
        return result
    
    response = llm.invoke([HumanMessage(content=user_input)])
    response_text = response.content
    print(f"LLM Response: {response_text[:100]}...")

    try:
        guard.validate(response_text)
        print('Output passed PII Check')
    except Exception as ex:
        result = 'OUTPUT BLOCKED: Contains PII'
        print(result)
        return result
    
    result = f"APPROVED: {response_text}"
    print(result)
    return result

In [ ]:
call_llm_guardrails('What is the weather of Bangalore')
call_llm_guardrails('Can you tell me few stupidity of AI')
call_llm_guardrails('Share few email ID o business analyst.')